In [ ]:
from PIL import Image
import numpy
import random
import math
import os

# mutation percentage
mp = 0.05

# swapping probability
sp = 0.5

# parameters of image
filename = "mitsu.png"
im = Image.open(filename)
width, height = im.size
channels = 3
chrom_length = width * height * channels


# data representation
def get_pixels(image_path):
    image = Image.open(image_path, 'r')
    width, height = image.size
    pixel_values = list(image.getdata())
    if image.mode == 'RGB':
        channels = 3
    elif image.mode == 'L':
        channels = 1
    else:
        print("Unknown mode: %s" % image.mode)
        return None
    pixel_list = [i for sets in pixel_values for i in sets]
    return pixel_list


def get_image(pixel_list):
    pixel_list = numpy.reshape(pixel_list, (height, width, channels))
    image = Image.fromarray(pixel_list.astype(numpy.uint8))
    return image
       

# initial population
def create_population(chrom_num=8):
    population = []
    for i in range(0, chrom_num):
        chromosome = numpy.random.random(chrom_length)*256
        population.append(chromosome)
    return population


# target image
file_name, file_extension = os.path.splitext(filename)
target = get_pixels(filename)
# print("Target fitness value is {0}".format(numpy.sum(target)))


def fitness_func(indiv_chrom, target_chrom=target):
     quality = numpy.mean(numpy.abs(target_chrom-indiv_chrom))
     quality = numpy.sum(target_chrom) - quality
     return quality
    
# frequency = 2000
def selection(population, i):
    n = len(population)
    fitness = [fitness_func(chromosome) for chromosome in population]
    print("Iteration: ", i, " fitness: ", numpy.max(fitness))
    idxs = numpy.argsort(fitness)
    idx = idxs[-1]
#     if (i % frequency == 0 and i != 0):
#             save_image(population[idx], i)
    idxs = idxs[int(n/2):]
    parents = [population[i] for i in idxs]
    children = []
    while (len(children)<int(n/2)):
        child1, child2 = crossover(parents[random.randint(0, int(n/2)-1)], parents[random.randint(0, int(n/2)-1)])
        child1, child2 = mutation(child1, mp), mutation(child2, mp)
        children.append(child1)
        children.append(child2)
        
    parents.extend(children)
    new_population = parents.copy()
    new_population = new_population[:n]
    
    return new_population
        

# crossover
def crossover(p1, p2, num_p=1, uniform=False):
    n = len(p1)
    child1, child2 = p1.copy(), p2.copy()
    
#     if uniform:
#         for i in range(n):
#             if random.uniform(0, 1) > sp:
#                 child1[i] = p2[i]
#                 child2[i] = p1[i]
#         return child1, child2
    
    half_size = int(n/2)
    child1[0:half_size] = p2[0:half_size]
    child2[0:half_size] = p1[0:half_size]
    return child1, child2


def mutation(chromosome, mut_percent):
    rand_idx = numpy.uint32(numpy.random.random(size=numpy.uint32(mut_percent/100*chrom_length))*chrom_length)
    new_values = numpy.uint8(numpy.random.random(size=rand_idx.shape[0])*256)
    chromosome[rand_idx] = new_values
    return chromosome


def print_init(population):
    fitness = [fitness_func(chromosome) for chromosome in population]
    idx = numpy.argsort(fitness)
    idx = idx[-1]
    print("Initial max finess value is {0}".format(fitness[idx]))
    i = get_image(population[idx])
    
#     print("It's image is\n")
#     i.show()

    os.remove("init.png")
    i.save("init.png")
    
    
def print_final(fitness, population):
    print("Max fitness value after evolution is {0}".format(fitness))
    i = get_image(population)
    
#     print("It's image is\n")
#     i.show()

    os.remove("final.png")
    i.save("final.png")
    
    
# def save_image(population, iteration):
#     i = get_image(population)
#     i.save("{0}_Iteration_{1}_exp.png".format(file_name, iteration))


# Genetic Algorithm
def ga(iter_num=20000):
    population = create_population(4)
    print_init(population)
    
    for i in range(0, iter_num):
        population = selection(population, i)
        
    fitness = [fitness_func(chromosome) for chromosome in population]
    idx = numpy.argsort(fitness)
    idx = idx[-1]
    return fitness[idx], population[idx]


f, p = ga()
print_final(f,p)